#Definition of used classes

In [ ]:
%pip install pygame

In [ ]:
import os
import math
from abc import ABC, abstractmethod
from typing import Any, Dict, List, Tuple
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow

from IPython.display import clear_output
from IPython.display import Video

from pygame import Vector2 as Vec2
from pygame import Color as Color

# Constants
FPS: int = 30
SCREEN_SIZE: Vec2 = Vec2(640, 440)
PITCH_SIZE: Vec2 = Vec2(600, 400)
PITCH_POS: Vec2 = (SCREEN_SIZE - PITCH_SIZE) / 2
POST_TOP_Y: float = PITCH_POS.y + PITCH_SIZE.y * 6 / 16
POST_BOT_Y: float = PITCH_POS.y + PITCH_SIZE.y * 10 / 16
LINE_WIDTH: int = 2

# Color palette
COLOR_WHITE: Color = Color(255, 255, 255)
COLOR_BLACK: Color = Color(0, 0, 0)
COLOR_LINE: Color = Color(174, 202, 137)
COLOR_PITCH: Color = Color(113, 152, 63)
COLOR_BACKGROUND: Color = Color(91, 143, 164)


class Drawable(ABC):
    @abstractmethod
    def draw(self, img):
        raise NotImplementedError()


class CirclePhysical(ABC):
    COUNT: int = 0

    def __init__(self, radius: float, mass: float, pos: Vec2, hooked: bool = False):
        self.id: int = CirclePhysical.COUNT
        CirclePhysical.COUNT += 1

        self.radius: float = radius
        self.mass: float = mass
        self.friction: float = 0.7 * mass
        self.v_max = 12 / math.pow(mass, 2 / 3)
        self.pos: Vec2 = pos
        self.vel: Vec2 = Vec2(0, 0)
        self.hooked: bool = hooked

    def setPos(self, pos: Vec2):
        self.pos = pos

    def setVel(self, vel: Vec2):
        self.vel = vel * self.mass

    def addVel(self, vel: Vec2):
        self.vel += vel

    def update(self, dt: float):
        self.pos += self.vel * dt

        self.vel += -self.vel * self.friction * dt
        self.pos += self.vel * dt

        if self.vel.magnitude() > self.v_max:
            self.vel = self.vel.normalize() * self.v_max

    @staticmethod
    def sphereCollisionVelocities(
            v1: Vec2, v2: Vec2,
            m1: float, m2: float,
            x1: Vec2, x2: Vec2) -> Tuple[Vec2, Vec2]:
        # Calculate new velocities after collision
        mass: float = 2 * m1 / (m1 + m2)
        v1_new: Vec2 = v1 - (mass * (v1 - v2).dot(x1 - x2) / pow((x1 - x2).length(), 2)) * (x1 - x2)
        v2_new: Vec2 = v2 - (mass * (v2 - v1).dot(x2 - x1) / pow((x2 - x1).length(), 2)) * (x2 - x1)
        return v1_new, v2_new

    def collide(self, other: 'CirclePhysical'):
        dist = self.pos.distance_to(other.pos)
        if dist < self.radius + other.radius:
            if dist == 0:
                self.pos += -self.vel.normalize() * 0.01

            self.vel, other.vel = CirclePhysical.sphereCollisionVelocities(
                self.vel, other.vel, self.mass, other.mass, self.pos, other.pos)

            if not self.hooked:
                self.pos += (self.pos - other.pos).normalize() * (self.radius + other.radius - dist) / 2
            else:
                self.vel = Vec2(0, 0)

            if not other.hooked:
                other.pos += (other.pos - self.pos).normalize() * (self.radius + other.radius - dist) / 2
            else:
                other.vel = Vec2(0, 0)


class Ball(CirclePhysical, Drawable):
    COLOR: Color = Color(255, 255, 255)
    MASS: float = 0.2
    RADIUS: float = 7

    def __init__(self, pos: Vec2 = Vec2(SCREEN_SIZE.x / 2, SCREEN_SIZE.y / 2)):
        super().__init__(
            radius=self.RADIUS, mass=self.MASS, pos=pos)

    def draw(self, img):
        cv2.circle(img, (int(self.pos.x), int(self.pos.y)), self.radius, self.COLOR, -1)


class Player(CirclePhysical, Drawable):
    MASS: float = 0.5
    RADIUS: float = 10

    def __init__(self, pos: Vec2, color: Color):
        super().__init__(
            radius=self.RADIUS, mass=self.MASS, pos=pos)

        self.color: Color = color

    def draw(self, img):
        cv2.circle(img, (int(self.pos.x), int(self.pos.y)), self.radius, self.color, -1)


class Post(CirclePhysical, Drawable):
    MASS: float = 1
    RADIUS: float = 6

    def __init__(self, pos: Vec2, color: Color):
        super().__init__(
            radius=self.RADIUS, mass=self.MASS, pos=pos, hooked=True)

        self.color: Color = color

    def draw(self, img):
        cv2.circle(img, (int(self.pos.x), int(self.pos.y)), self.radius, self.color, -1)


class Goal(Drawable):
    def __init__(self, dir: int, linePos: float, postTopY: float, postBotY: float, color: Color):
        self.dir: int = dir
        self.linePos: float = linePos
        self.yTop: float = postTopY
        self.yBot: float = postBotY
        self.color: Color = color

        self.postTop: Post = Post(Vec2(linePos, postTopY), color)
        self.postBot: Post = Post(Vec2(linePos, postBotY), color)

    def draw(self, img):
        cv2.line(img, (int(self.linePos), int(self.yTop)), (int(self.linePos), int(self.yBot)), self.color, LINE_WIDTH)
        self.postTop.draw(img)
        self.postBot.draw(img)


class Team(Drawable):
    def __init__(self, dir: int, color: Color):
        self.color: Color = color

        # Dir is 1 or -1
        # 1 is attacking from left to right
        # -1 is attacking from right to left
        self.dir: int = dir

        # Normalized dir is 0 or 1
        # 0 is attacking from left to right
        # 1 is attacking from right to left
        self.normalizedDir: int = -(dir - 1) / 2

        # Goal line for dir 1 is pitchPos.x
        # Goal line for dir -1 is pitchPos.x + pitchSize.x
        goalLineX: float = PITCH_POS.x + PITCH_SIZE.x * self.normalizedDir

        self.goal: Goal = Goal(dir, goalLineX, POST_TOP_Y, POST_BOT_Y, color)
        self.player: Player = Player(Vec2(goalLineX, SCREEN_SIZE.y / 2), color)

        self.score: int = 0

    def draw(self, img):
        self.goal.draw(img)
        self.player.draw(img)

    def reset(self):
        self.score = 0
        self.player.setPos(Vec2(self.goal.linePos + self.dir * PITCH_SIZE.x / 4, SCREEN_SIZE.y / 2 + self.dir * SCREEN_SIZE.y / 6))
        self.player.setVel(Vec2(0, 0))

    def applyAction(self, action: Vec2):
        # Convert action to direction and invert it in relation to team dir
        if action.length() != 0:
            vel: Vec2 = action.normalize() * self.dir

            # Apply velocity to player
            self.player.addVel(vel)


class Pitch(Drawable):
    def __init__(self):
        self.team1: Team = Team(1, Color(255, 0, 0))
        self.team2: Team = Team(-1, Color(0, 0, 255))

    def draw(self, img):
        # Draw pitch as a solid color filled rectangle with a line border
        cv2.rectangle(img, (int(PITCH_POS.x), int(PITCH_POS.y)),
                      (int(PITCH_POS.x + PITCH_SIZE.x), int(PITCH_POS.y + PITCH_SIZE.y)), COLOR_PITCH, -1)
        cv2.rectangle(img, (int(PITCH_POS.x), int(PITCH_POS.y)),
                      (int(PITCH_POS.x + PITCH_SIZE.x), int(PITCH_POS.y + PITCH_SIZE.y)), COLOR_LINE, LINE_WIDTH)

        # Draw center line
        cv2.line(img, (int(PITCH_POS.x + PITCH_SIZE.x / 2), int(PITCH_POS.y)),
                 (int(PITCH_POS.x + PITCH_SIZE.x / 2), int(PITCH_POS.y + PITCH_SIZE.y)), COLOR_LINE, LINE_WIDTH)
        cv2.circle(img, (int(PITCH_POS.x + PITCH_SIZE.x / 2), int(PITCH_POS.y + PITCH_SIZE.y / 2)),
                   int(PITCH_SIZE.y / 6), COLOR_LINE, LINE_WIDTH)

        # Draw goals
        self.team1.draw(img)
        self.team2.draw(img)

    def pitchCollision(self, sphere: CirclePhysical, dt: float) -> int:
        # Check sphere's collision with pitch
        # Return 0 if nothing happens
        # Return 1 if team sphere is in goal of team 1
        # Return 2 if team sphere is in goal of team 2

        # Check if the sphere collides with the pitch's top or bottom
        if (sphere.pos.y - sphere.radius < PITCH_POS.y or
                sphere.pos.y + sphere.radius > PITCH_POS.y + PITCH_SIZE.y):
            sphere.pos.y += -sphere.vel.y * dt
            sphere.vel.y *= -1

        # Check if the sphere collides with the goal
        if (sphere.pos.y > POST_TOP_Y and sphere.pos.y < POST_BOT_Y) and isinstance(sphere, Ball):
            # Check if the sphere collides with the goal of team 1
            if (sphere.pos.x + sphere.radius < self.team1.goal.linePos):
                return 1

            # Check if the sphere collides with the goal of team 2
            if (sphere.pos.x - sphere.radius > self.team2.goal.linePos):
                return 2

            return 0

        # Check if the sphere collides with the pitch's left or right
        if (sphere.pos.x - sphere.radius < PITCH_POS.x or
                sphere.pos.x + sphere.radius > PITCH_POS.x + PITCH_SIZE.x):
            sphere.pos.x += -sphere.vel.x * dt
            sphere.vel.x *= -1

        return 0


class ReplayBuffer():
    def __init__(self):
        self.buffer: List[Tuple(Vec2, Vec2, Vec2)] = []

    def snapshot(self, ball: Ball, team1: Team, team2: Team):
        self.buffer.append((ball.pos.copy(), team1.player.pos.copy(), team2.player.pos.copy()))

    def reset(self):
        self.buffer = []


class Haxball():
    def __init__(self):
        self.ball: Ball = Ball()
        self.pitch: Pitch = Pitch()
        self.player1: Player = self.pitch.team1.player
        self.player2: Player = self.pitch.team2.player

        self.circlePhysicals: List[CirclePhysical] = [
            self.ball, self.player1, self.player2,
            self.pitch.team1.goal.postTop, self.pitch.team1.goal.postBot,
            self.pitch.team2.goal.postTop, self.pitch.team2.goal.postBot
        ]

        self.paused: bool
        self.lastAction1: Vec2 = Vec2(0, 0)
        self.lastAction2: Vec2 = Vec2(0, 0)

        self.replayBuffer: ReplayBuffer = ReplayBuffer()

        self.reset()

    def reset(self):
        self.paused = False
        self.lastAction1 = Vec2(0, 0)
        self.lastAction2 = Vec2(0, 0)

        # Reset ball
        self.ball.setPos(Vec2(SCREEN_SIZE.x / 2, SCREEN_SIZE.y / 2))
        self.ball.setVel(Vec2(0, 0))

        # Reset teams
        self.pitch.team1.reset()
        self.pitch.team2.reset()

        # Save first snapshot to replay buffer
        self.replayBuffer.reset()
        self.replayBuffer.snapshot(self.ball, self.pitch.team1, self.pitch.team2)

    @staticmethod
    def getRelativePos(pos: Vec2, dir: int) -> Vec2:
        # Return player position normalized to the given dir
        # (0, PITCH_SIZE.y / 2) is the center of own goal line

        normalizedDir = -(dir - 1) / 2

        return (pos - SCREEN_SIZE * normalizedDir) * dir

    @staticmethod
    def getRelativeVel(vel: Vec2, dir) -> Vec2:
        # Return player velocity normalized to the given dir

        return vel * dir

    def info(self):
        info = dict()
        info["pitch_size"] = (int(PITCH_SIZE.x), int(PITCH_SIZE.y))
        info["pitch_pos"] = (int(PITCH_POS.x), int(PITCH_POS.y))
        info["screen_size"] = (int(SCREEN_SIZE.x), int(SCREEN_SIZE.y))
        info["player_radius"] = Player.RADIUS
        info["ball_radius"] = Ball.RADIUS
        info["post_radius"] = Post.RADIUS
        info["goal_width"] = POST_BOT_Y - POST_TOP_Y
        info["own_postTop_pos"] = self.getRelativePos(self.pitch.team1.goal.postTop.pos, 1)
        info["own_postBot_pos"] = self.getRelativePos(self.pitch.team1.goal.postBot.pos, 1)
        info["opp_postTop_pos"] = self.getRelativePos(self.pitch.team2.goal.postTop.pos, 1)
        info["opp_postBot_pos"] = self.getRelativePos(self.pitch.team2.goal.postBot.pos, 1)
        info["state_space"] = 12
        info["action_space"] = 2
        return info

    def state(self):
        return self.getState(1), self.getState(2), self.paused

    def render(self):
        img = np.zeros((int(SCREEN_SIZE.y), int(SCREEN_SIZE.x), 3), np.uint8)

        cv2.rectangle(img, (0, 0), (int(SCREEN_SIZE.x), int(SCREEN_SIZE.y)), COLOR_BACKGROUND, -1)

        self.pitch.draw(img)
        self.ball.draw(img)

        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        return img

    def showReplay(self):
        video_path = "tmp.mp4"

        video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"JPEG"), FPS, (self.render().shape[1::-1]))

        # Create video for whole replay buffer
        for snapshot in self.replayBuffer.buffer:
            # print(snapshot)
            self.ball.pos = snapshot[0]
            self.pitch.team1.player.pos = snapshot[1]
            self.pitch.team2.player.pos = snapshot[2]

            video.write(self.render())

        video.release()
        fileToDisplay = "video.mp4"
        os.system(f"ffmpeg -y -i {video_path} -vcodec libx264 -x264opts keyint=123:min-keyint=120 -an {fileToDisplay}")
        display(Video(fileToDisplay, embed=True))

    def getState(self, team: int):
        dir: int
        reward: int
        lastAction: Vec2
        player: Player
        opponent: Player

        if team == 1:
            dir = 1
            reward = self.pitch.team1.score
            lastAction = self.lastAction1
            player = self.player1
            opponent = self.player2
        elif team == 2:
            dir = -1
            reward = self.pitch.team2.score
            lastAction = self.lastAction2
            player = self.player2
            opponent = self.player1
        else:
            raise Exception("Invalid team")

        ballPos: Vec2 = self.getRelativePos(self.ball.pos, dir)
        ballVel: Vec2 = self.getRelativeVel(self.ball.vel, dir)
        playerPos: Vec2 = self.getRelativePos(player.pos, dir)
        playerVel: Vec2 = self.getRelativeVel(player.vel, dir)
        opponentPos: Vec2 = self.getRelativePos(opponent.pos, dir)
        opponentVel: Vec2 = self.getRelativeVel(opponent.vel, dir)

        state: List[float] = [
            ballPos.x, ballPos.y, ballVel.x, ballVel.y,
            playerPos.x, playerPos.y, playerVel.x, playerVel.y,
            opponentPos.x, opponentPos.y, opponentVel.x, opponentVel.y
        ]

        return state, reward, lastAction

    def collide(self, sphere: CirclePhysical, dt: float) -> int:
        # Collide with other spheres
        for other in self.circlePhysicals:
            if other != sphere:
                sphere.collide(other)

        # Collide with pitch
        return self.pitch.pitchCollision(sphere, dt)

    def step(self, action1: Vec2, action2: Vec2):
        if self.paused:
            self.replayBuffer.snapshot(self.ball, self.pitch.team1, self.pitch.team2)
            return self.state()

        dt: float = 15 / FPS

        # Apply actions
        self.pitch.team1.applyAction(action1)
        self.pitch.team2.applyAction(action2)

        # Update players
        self.player1.update(dt)
        self.player2.update(dt)
        # Update ball
        self.ball.update(dt)

        # Check for collisions
        self.collide(self.player1, dt)
        self.collide(self.player2, dt)
        res: int = self.collide(self.ball, dt)

        # Save snapshot to replay buffer
        self.replayBuffer.snapshot(self.ball, self.pitch.team1, self.pitch.team2)

        self.lastAction1 = action1
        self.lastAction2 = action2

        # Check if a goal was scored
        if res == 1:
            print("GOAL! for team 2")
            self.pitch.team2.score += 1
            self.paused = True
        elif res == 2:
            self.pitch.team1.score += 1
            print("GOAL! for team 1")
            self.paused = True

        return self.state()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


# The environment
The environment was defined in the Haxball class.<br>

### State
State is returned by function `step(action1, action2)` or `state()`.
State consists of 3 values: `state1, state2, done`:
* `state1` -- `Tuple(state, reward, lastAction)`:
  * `state` -- `Dict`: game state from the perspective of a player 1:
    * `ballPos` -- `Vec2`: position of the ball
    * `ballVel` -- `Vec2`: velocity of the ball
    * `playerPos` -- `Vec2`: position of the player
    * `playerVel` -- `Vec2`: velocity of the player
    * `opponentPos` -- `Vec2`: position of the opponent
    * `opponentVel` -- `Vec2`: velocity of the opponent
  * `reward` -- `int`: reward for the last action
  * `lastAction` -- `Vec2`: last action taken by the player
* `state2` -- same as `state1` but from the perspective of a player 2
* `done` -- `bool`: True if the game is over (goal was scored)


### Info
Function `info()` returns a dictionary containing information about the environment:
 * `pitch_size` -- size of the pitch
 * `pitch_pos` -- position of the top left corner of the screen
 * `screen_size` -- size of the whole screen (pitch + border)
 * `player_radius` -- radius of the player
 * `ball_radius` -- radius of the ball
 * `post_radius` -- radius of the post
 * `goal_width` -- width of the goal (y of the bottom post - y of the top post)

### Action
Action is continuous and is a `Vec2` type. It specifies the direction of the movement. It doesn't have to be normalized (magnitude doesn't affect the velocity).

Action is relative to the player's goal. `Vec(1, 0)` (forward movement along x-axis) for one player means going to the right side of the screen, but for another it means going to the left (from the viewer's perspective).

States are already properly reflected in relation to player's goal, so if you work on them, actions should work the some for both players - there is no difference which player you are using.

`Vec2` is a `pygame.Vector2` type. You can look up it's documentation online. It supports basic vector operations.


### Reset
`reset()` resets the environment: score, all positions and velocities and replay buffer.

### Replays
`showReplay()` displays video of a last game (since the last reset).

### Environment parameters
On top of the code "Constants" section can be found. They can be used to modify the environment.
You can change them to modify the environment:
* `FPS` - number of frames per second - affects the video, but also the simulation (changes number of simulation steps)
* `SCREEN_SIZE` - size of the screen - can't be smaller than pitch size - affects performance of video rendering
* `PITCH_SIZE` - size of the pitch - affects the domain of the problem
* `POST_TOP_Y/POST_BOT_Y` - width of the goal


In [ ]:
env = Haxball()
info = env.info()
print(f"info: {info}\n")
state1, state2, done = env.state()
print(f"state1: {state1}\n")
print(f"state2: {state2}\n")
state1, state2, done = env.step(Vec2(0, 1), Vec2(0, 1))
state1, state2, done = env.state()
print(f"state1: {state1}\n")
print(f"state2: {state2}\n")
print(f"done:  {done}")


info: {'pitch_size': (600, 400), 'pitch_pos': (20, 20), 'screen_size': (640, 440), 'player_radius': 10, 'ball_radius': 7, 'post_radius': 6, 'goal_width': 100.0, 'own_postTop_pos': <Vector2(20, 170)>, 'own_postBot_pos': <Vector2(20, 270)>, 'opp_postTop_pos': <Vector2(620, 170)>, 'opp_postBot_pos': <Vector2(620, 270)>, 'state_space': 12, 'action_space': 2}

state1: ([320.0, 220.0, 0.0, 0.0, 170.0, 293.3333333333333, 0.0, 0.0, 470.0, 146.66666666666669, 0.0, 0.0], 0, <Vector2(0, 0)>)

state2: ([320.0, 220.0, -0.0, -0.0, 170.0, 293.3333333333333, -0.0, -0.0, 470.0, 146.66666666666669, -0.0, -0.0], 0, <Vector2(0, 0)>)

state1: ([320.0, 220.0, 0.0, 0.0, 170.0, 294.24583333333334, 0.0, 0.825, 470.0, 145.7541666666667, 0.0, -0.825], 0, <Vector2(0, 1)>)

state2: ([320.0, 220.0, -0.0, -0.0, 170.0, 294.2458333333333, -0.0, 0.825, 470.0, 145.75416666666666, -0.0, -0.825], 0, <Vector2(0, 1)>)

done:  False


## Sample game simulation

Below you can see a sample game simulation. Half of the moves are random and half are forward (notice that forward movement is different direction for each player!).

Frames count is specified in `frames` variable. You can change it to see longer games. Length of the game (in seconds) is `frames / FPS`.

After the game, replay is generated.

In [ ]:
import random

def randomVector():
    # Return a random vector from -1 to 1
    return Vec2(random.random() * 2 - 1, random.random() * 2 - 1)

env.reset()

# Play a random game
frames: int = 720

for i in range(frames):
    if random.random() < 0.5:
        # Move randomly
        state1, state2, done = env.step(randomVector(), randomVector())
    else:
        # Move forward
        state1, state2, done = env.step(Vec2(15, 0), Vec2(1, 0))

env.showReplay()

## Movement towards the ball

Below you can see a different game simulation. In this game, players are always moving towards the ball. Notice how the direction towards the ball is calculated from state info.

In [ ]:
import random

env = Haxball()

# Play a random game
frames: int = 1800

for i in range(frames):
    # Move agent towards the ball
    player1State, reward, lastAction = env.getState(1)
    player2State, reward, lastAction = env.getState(2)

    ball1Pos: Vec2 = Vec2(player1State[0], player1State[1])
    player1Pos: Vec2 = Vec2(player1State[4], player1State[5])

    ball2Pos: Vec2 = Vec2(player2State[0], player2State[1])
    player2Pos: Vec2 = Vec2(player2State[4], player2State[5])

    diff1: Vec2 = (ball1Pos - player1Pos).normalize()
    diff2: Vec2 = (ball2Pos - player2Pos).normalize()

    state1, state2, done = env.step(diff1, diff2)


env.showReplay()

GOAL! for team 2


#Zadanie wykładowe 2
Stwórz wirtualnego gracza piłki nożnej sterowanego z wykorzystaniem sieci neuronowej. Wykorzystaj jeden z algortmów z wykładu siódmego: DQN, Double DQN lub Dueling DQN.

#Zadania do wykonania
* Opracuj sieć neuronową wyznaczającą wartość Q.
* Zaimplementuj algorytm uczenia sieci neuronowej.
* Naucz piłkarza dryblowania, czyli prowadzenia piłki do wskazanego punktu.
* Naucz piłkarza omijania przeciwnika i prowadzenia piłki do bramki.
* Przeprowadź gry między opracowanymi graczami.



In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from pygame import Vector2 as Vec2

DISCRETE_ACTIONS = [
    Vec2(0, 0),      # no-op
    Vec2(1, 0),      # right
    Vec2(-1, 0),     # left
    Vec2(0, 1),      # down
    Vec2(0, -1),     # up
    Vec2(1, 1),      # right-down
    Vec2(1, -1),     # right-up
    Vec2(-1, 1),     # left-down
    Vec2(-1, -1),    # left-up
]
NUM_ACTIONS = len(DISCRETE_ACTIONS)

class DqnReplayBuffer:
    def __init__(self, capacity=50000):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (np.array(states, dtype=np.float32),
                np.array(actions, dtype=np.int64),
                np.array(rewards, dtype=np.float32),
                np.array(next_states, dtype=np.float32),
                np.array(dones, dtype=np.bool_))

    def __len__(self):
        return len(self.buffer)

class DQN(nn.Module):
    def __init__(self, state_dim=12, action_dim=9, hidden_dim=128):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),  # Add Layer Normalization
            nn.ReLU(),
            nn.Linear(hidden_dim, 2 * hidden_dim),
            nn.LayerNorm(2 * hidden_dim),  # Add Layer Normalization
            nn.ReLU(),
            nn.Linear(2 * hidden_dim, 2 * hidden_dim),
            nn.LayerNorm(2 * hidden_dim),  # Add Layer Normalization
            nn.ReLU(),
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),  # Add Layer Normalization
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

    def forward(self, x):
        return self.net(x)

class DQNAgent:
    def __init__(self,
                 state_dim=12,
                 action_dim=9,
                 lr=1e-3,
                 gamma=0.99,
                 replay_size=50000,
                 batch_size=64,
                 device='cpu'):
        self.device = device
        self.gamma = gamma
        self.batch_size = batch_size

        self.dqn = DQN(state_dim, action_dim).to(device)
        self.target_dqn = DQN(state_dim, action_dim).to(device)
        self.target_dqn.load_state_dict(self.dqn.state_dict())

        self.optimizer = optim.Adam(self.dqn.parameters(), lr=lr)

        self.replay_buffer = DqnReplayBuffer(capacity=replay_size)

        self.epsilon = 1.0

    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, NUM_ACTIONS - 1)
        else:
            with torch.no_grad():
                s_t = torch.FloatTensor(state).unsqueeze(0).to(self.device)
                q_values = self.dqn(s_t)  # shape: [1, action_dim]
                action = q_values.argmax(dim=1).item()
                return action

    def update_target_net(self):
        self.target_dqn.load_state_dict(self.dqn.state_dict())

    def store_transition(self, s, a, r, s_next, done):
        self.replay_buffer.push(s, a, r, s_next, done)

    def train_step(self):

        # One gradient update from a sampled batch of the replay buffer.

        if len(self.replay_buffer) < self.batch_size:
            return  # not enough data

        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)

        states = torch.FloatTensor(states).to(self.device)        # [B, 12]
        actions = torch.LongTensor(actions).to(self.device)       # [B]
        rewards = torch.FloatTensor(rewards).to(self.device)      # [B]
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.BoolTensor(dones).to(self.device)           # [B]

        q_values = self.dqn(states)          # [B, action_dim]
        q_a = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)  # [B]

        with torch.no_grad():
            q_next = self.target_dqn(next_states)                 # [B, action_dim]
            max_q_next = q_next.max(dim=1)[0]                     # [B]
            # Q target
            q_target = rewards + self.gamma * max_q_next * (~dones)

        loss = nn.MSELoss()(q_a, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

def distance_from_goal_reward(
    team_id: int,
    env,
    new_state: np.ndarray,
    old_score: int,
    new_score: int,
    verbose: bool
) -> float:
    reward = 0.0

    if new_score > old_score:
        reward += 10.0


    opp_goal_x = env.pitch.team2.goal.linePos
    goal_y_center = (POST_TOP_Y + POST_BOT_Y) / 2.0

    new_ball_x, new_ball_y = new_state[0], new_state[1]

    new_player_x, new_player_y = new_state[4], new_state[5]

    new_dist_goal = np.sqrt((opp_goal_x - new_ball_x)**2 + (goal_y_center - new_ball_y)**2)
    reward +=  (300 - new_dist_goal)/300

    new_dist_player_ball = np.sqrt((new_player_x - new_ball_x)**2 + (new_player_y - new_ball_y)**2)
    # sqrt(600**2 + 400**2) - 17
    reward -= (((new_dist_player_ball) / 721))

    # print(f"Goal distance: old({old_dist_goal}), new({new_dist_goal}). Ball distance: old({old_dist_player_ball}), new({new_dist_player_ball})")
    if verbose:
      print(f"Goal distance: new({new_dist_goal}). Ball distance: new({new_dist_player_ball}). Player position: ({new_player_x}, {new_player_y}). Reward: {reward}")
    return reward


def validate(agent1_path, agent2_path, eps1, eps2):
    agent1 = DQNAgent(state_dim=12, action_dim=NUM_ACTIONS, device=device)
    agent2 = DQNAgent(state_dim=12, action_dim=NUM_ACTIONS, device=device)
    agent1.dqn.load_state_dict(torch.load(f"{agent1_path}_agent1.pth", map_location=device))
    agent2.dqn.load_state_dict(torch.load(f"{agent2_path}_agent2.pth", map_location=device))
    agent1.target_dqn.load_state_dict(agent1.dqn.state_dict())
    agent2.target_dqn.load_state_dict(agent2.dqn.state_dict())
    agent1.epsilon = eps1
    agent2.epsilon = eps2


    print(f"\n\n{agent1_path}\n{agent2_path}\n\n")
    for e in range(test_episodes):
      env.reset()
      done = False
      (state1, _, _), (state2, _, _), done = env.state()
      state1 = np.array(state1, dtype=np.float32)
      state2 = np.array(state2, dtype=np.float32)
      steps = 0
      r1 = 0
      r2 = 0
      while not done and steps < 500:
          steps += 1
          a1 = agent1.select_action(state1)
          a2 = agent2.select_action(state2)
          (next_state1, _, _), (next_state2, _, _), done = env.step(DISCRETE_ACTIONS[a1], DISCRETE_ACTIONS[a2])
          state1 = np.array(next_state1, dtype=np.float32)
          state2 = np.array(next_state2, dtype=np.float32)
          r1 += distance_from_goal_reward(1, env, state1, 0, env.pitch.team1.score, False)
          r2 += distance_from_goal_reward(2, env, state2, 0, env.pitch.team2.score, False)
      print(f"R1={r1/steps}, R2={r2/steps}")
      env.showReplay()

if __name__ == "__main__":
    env = Haxball()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    test_episodes = 5

    validate("pretrained", "pretrained", 0.01, 1.0)
    validate("pretrained", "pretrained", 1.0, 0.01)
    validate("trained", "trained", 0.01, 0.01)
    validate("trained", "pretrained", 0.01, 0.01)
    validate("pretrained", "trained", 0.01, 0.01)


print("Done testing!")



<ipython-input-14-5652c544b178>:170: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  agent1.dqn.load_state_dict(torch.load(f"{agent1_path}_agent1.pth", map_location=device))
<



pretrained
pretrained


GOAL! for team 1
R1=0.6556614408418233, R2=-0.8930103547808553


GOAL! for team 1
R1=0.38781965862135576, R2=-0.5424267547380075


GOAL! for team 1
R1=0.3816648621477685, R2=-0.5205162228763845


GOAL! for team 1
R1=0.3880405803938607, R2=-0.5809991283417015


...

R1=-0.5717761363879811, R2=0.2856230920945142


R1=-0.1903454315446749, R2=-0.16440809563871484


R1=-0.2029683456803284, R2=-0.16021171989996832


R1=-0.17954185625857177, R2=-0.17326516111405355


Done testing!
